In [14]:
import copy
import random
class node:
  def __init__(self,state,h,g,f,fp, parent,zero,w):
    #node = (state,h,g,f,fp, parent,zero)
    name = [item for sublist in state for item in sublist] 
    self.name = "".join([str(int) for int in name])
    self.state = state
    self.hval = h
    self.gval = g
    self.fval = f
    self.whval = w*h
    self.fpval = fp
    self.parent = parent
    self.zero = zero



class EightPuzzle:
  def __init__(self,start,goal,w,p):
    self.start = self.to2D(start)
    self.goal = self.to2D(goal)
    self.w = w
    self.p = p
    self.counter = 0
    self.solved = False
    self.open = {}
    self.closed = {}
    #node = (state,h,g,f,fp, parent)
    h = self.mh(self.start)
    g = 0
    f = h+g
    fp = self.w*h + g
    start = node(self.start,h,g,f,fp,None,self.index_2d(self.start,0),self.w)
    self.open[start.name] = start

  def addChild(self,children):
    for child in children:
      if (child.name not in self.open and child.name not in self.closed):
        self.open[child.name] = child
      else:
        if child.name in self.open:
          if (self.open[child.name].gval > child.gval):
            self.open[child.name].gval = child.gval
            self.open[child.name].parent = child.parent
 
            
        elif child.name in self.closed:
          if (self.closed[child.name].gval > child.gval):
            self.open[child.name] = child
            del self.closed[child.name]


  def expand(self, current_node):
    children = self.getChildren(current_node)
    self.closed[current_node.name] = current_node
    del self.open[current_node.name]
    self.addChild(children)




  def getChildren(self,current_node):
    children = []
    parentCost = current_node.gval
    state = current_node.state
    blank = current_node.zero
    left = (blank[0]-1,blank[1])
    right = (blank[0]+1,blank[1])
    up = (blank[0],blank[1]+1)
    down  = (blank[0],blank[1]-1)
    valid = [(0,0),(0,1),(0,2),(0,3),(1,0),(1,1),(1,2),(1,3),(2,0),(2,1),(2,2),(2,3),(3,0),(3,1),(3,2),(3,3)]
    for move in [left,right, up, down]:
      if move in valid:
        temp = copy.deepcopy(state)
        temp[blank[0]][blank[1]] = temp[move[0]][move[1]]
        temp[move[0]][move[1]] = 0
        h = self.mh(temp)
        wh = self.w*h
        g = parentCost + 1
        #node = (state,h,g,f,fp, parent,zero)
        child = node(temp,h,g,g+h,g+wh,state,move,self.w)
        children.append(child)
    return children

  def to2D(self,state):
    state = [state[0:4],state[4:8],state[8:12],state[12:16]]
    return state

  def index_2d(self, myList, v):

    for i, x in enumerate(myList):
        if v in x:
            return (i, x.index(v))

  def mh(self,state):
    errors = []
    for i in range(1,16):
      s = self.index_2d(state,i)
      g = self.index_2d(self.goal,i)
      x = abs(s[0]-g[0])
      y = abs(s[1]-g[1])
      errors.append(x+y)
    return sum(errors)

  def path(self):
    path = []
    parent = 0
    i = -1
    closed_strip = list( map(itemgetter(0), self.closed))
    state = self.closed[i]
    #path.insert(0,state[0])
    path.append(state[0])
    parent = copy.deepcopy(state[4])
    while(parent != None):
      state = copy.deepcopy(parent)
      parent = self.closed[closed_strip.index(state)][4]
      path.insert(0,state)
      

    return path

  def solve(self):
    while(not self.solved):
      self.counter = self.counter + 1
      if self.counter % self.p == 0:
        current_node = min(self.open.values(), key=lambda x: x.fpval)
      else: 
        current_node = min(self.open.values(), key=lambda x: x.fval)
        focal = dict((k, v) for k, v in self.open.items() if v.fval <= self.w*current_node.fval)
        current_node = min(focal.values(), key=lambda x: x.whval)
      if current_node.hval == 0:
        #self.closed.append(self.open[0])
        del self.open[current_node.name]
        self.closed[current_node.name] = current_node
        self.sovled = True
        break
      self.expand(current_node)
    return self.counter


In [15]:
starts =  [[14, 13, 15, 7 ,11, 12, 9 ,5 ,6 ,0 ,2 ,1 ,4 ,8 ,10 ,3],
[13, 5, 4 ,10 ,9 ,12 ,8 ,14, 2 ,3 ,7 ,1 ,0 ,15, 11 ,6],
[14 ,7 ,8 ,2 ,13, 11, 10, 4 ,9, 12, 5 ,0, 3, 6, 1 ,15 ],
[5 ,12 ,10 ,7 ,15 ,11 ,14 ,0 ,8 ,2 ,1 ,13 ,3 ,4 ,9 ,6 ],
[4 ,7, 14 ,13, 10, 3, 9, 12, 11, 5, 6, 15, 1, 2, 8, 0],
[14 ,7 ,1 ,9 ,12 ,3 ,6 ,15 ,8 ,11 ,2 ,5, 10, 0 ,4 ,13],
[2 ,11, 15, 5, 13 ,4, 6, 7, 12, 8, 10, 1, 9, 3 ,14, 0 ],
[12 ,11, 15 ,3 ,8 ,0 ,4 ,2 ,6 ,13 ,9 ,5 ,14, 1 ,10 ,7 ],
[3 ,14, 9, 11, 5 ,4, 8, 2, 13, 12, 6, 7 ,10, 1 ,15, 0],
[13 ,11 ,8 ,9 ,0 ,15 ,7 ,10 ,4 ,3 ,6 ,14 ,5 ,12 ,2 ,1],
[5 ,9 ,13 ,14 ,6 ,3, 7 ,12 ,10 ,8, 4 ,0 ,15, 2 ,11 ,1 ],
[14 ,1, 9, 6 ,4 ,8 ,12 ,5 ,7 ,2 ,3, 0 ,10, 11, 13, 15],
[3 ,6 ,5, 2, 10, 0, 15, 14, 1 ,4 ,13 ,12 ,9, 8 ,11, 7],
[7 ,6, 8, 1, 11, 5, 14, 10, 3 ,4 ,9, 13 ,15, 2, 0, 12 ],
[13 ,11 ,4 ,12 ,1 ,8 ,9 ,15 ,6 ,5 ,14 ,2 ,7 ,3 ,10 ,0 ],
[1 ,3, 2, 5 ,10, 9, 15, 6 ,8 ,14 ,13, 11 ,12, 4, 7, 0 ],
[15 ,14 ,0 ,4 ,11 ,1 ,6 ,13 ,7 ,5 ,8 ,9 ,3 ,2 ,10 ,12 ],
[6 ,0 ,14, 12 ,1 ,15, 9 ,10, 11, 4 ,7 ,2 ,8, 3, 5, 13],
[7 ,11 ,8, 3 ,14 ,0 ,6 ,15 ,1 ,4 ,13 ,9 ,5 ,12 ,2 ,10 ],
[6, 12, 11, 3 ,13, 7 ,9, 15, 2, 14, 8, 10, 4 ,1, 5, 0 ],
[12 ,8, 14 ,6, 11 ,4 ,7 ,0 ,5 ,1 ,10 ,15 ,3 ,13 ,9 ,2],
[14 ,3, 9 ,1, 15, 8 ,4 ,5, 11 ,7 ,10, 13, 0 ,2 ,12, 6],
[10 ,9 ,3, 11 ,0 ,13 ,2 ,14, 5 ,6 ,4 ,7 ,8 ,15 ,1 ,12 ],
[7 ,3, 14, 13 ,4 ,1 ,10, 8 ,5 ,12, 9, 11, 2 ,15, 6 ,0 ],
[11 ,4 ,2 ,7 ,1 ,0 ,10 ,15 ,6 ,9 ,14 ,8 ,3, 13 ,5 ,12 ],
[5 ,7, 3, 12 ,15 ,13 ,14, 8 ,0 ,10, 9 ,6 ,1 ,4, 2, 11 ],
[14 ,1 ,8 ,15 ,2 ,6 ,0 ,3 ,9 ,12 ,10 ,13 ,4 ,7 ,5 ,11 ],
[13 ,14, 6 ,12, 4 ,5, 1, 0, 9, 3, 10, 2, 15, 11, 8, 7 ],
[9 ,8 ,0 ,2 ,15 ,1, 4 ,14, 3 ,10 ,7, 5 ,11 ,13, 6, 12 ],
[12 ,15, 2 ,6, 1, 14, 4, 8, 5 ,3 ,7, 0, 10, 13, 9, 11 ],
[12 ,8 ,15 ,13, 1 ,0 ,5, 4 ,6 ,3 ,2 ,11 ,9 ,7 ,14 ,10 ],
[14 ,10, 9, 4, 13, 6, 5, 8 ,2, 12, 7, 0, 1, 3, 11, 15],
[14 ,3 ,5 ,15, 11 ,6 ,13, 9 ,0 ,10 ,2 ,12, 4 ,1 ,7, 8 ],
[6, 11, 7 ,8 ,13, 2, 5, 4 ,1, 10, 3, 9, 14, 0 ,12, 15 ],
[1 ,6 ,12, 14, 3, 2 ,15 ,8 ,4, 5 ,13 ,9, 0, 7, 11, 10 ],
[12, 6 ,0 ,4 ,7 ,3 ,15, 1, 13, 9, 8 ,11, 2, 14, 5, 10 ],
[8 ,1 ,7 ,12 ,11, 0 ,10, 5, 9, 15, 6 ,13 ,14, 2, 3, 4 ],
[7 ,15, 8, 2 ,13, 6, 3, 12, 11, 0 ,4, 10, 9, 5, 1, 14 ],
[9, 0 ,4 ,10 ,1 ,14 ,15 ,3 ,12 ,6, 5 ,7, 11, 13, 8, 2 ],
[11, 5, 1 ,14, 4 ,12, 10, 0 ,2 ,7, 13, 3, 9, 15, 6, 8 ],
[8 ,13 ,10 ,9 ,11, 3, 15 ,6 ,0, 1 ,2 ,14 ,12, 5, 4, 7 ],
[4 ,5, 7, 2, 9 ,14 ,12, 13 ,0 ,3, 6, 11, 8, 1, 15, 10 ],
[11 ,15 ,14 ,13 ,1 ,9 ,10, 4 ,3, 6, 2 ,12 ,7 ,5, 8, 0 ],
[12 ,9, 0, 6, 8, 3, 5, 14, 2, 4 ,11, 7 ,10, 1, 15, 13 ],
[3 ,14, 9 ,7 ,12 ,15, 0, 4, 1 ,8 ,5 ,6, 11 ,10 ,2, 13 ],
[8, 4, 6, 1, 14, 12, 2 ,15, 13, 10, 9, 5 ,3, 7, 0, 11 ],
[6 ,10 ,1 ,14 ,15, 8, 3 ,5 ,13 ,0 ,2, 7 ,4, 9, 11, 12 ],

[8 ,11, 4 ,6 ,7 ,3 ,10, 9, 2, 12, 15, 13, 0, 1, 5, 14],
[10 ,0 ,2, 4, 5, 1, 6, 12 ,11 ,13 ,9 ,7, 15, 3 ,14 ,8 ],
[12 ,5, 13 ,11, 2, 10, 0 ,9 ,7, 8, 4 ,3 ,14 ,6, 15, 1 ],
[10 ,2 ,8, 4 ,15, 0, 1, 14 ,11, 13 ,3 ,6, 9 ,7 ,5 ,12 ],
[10 ,8, 0 ,12, 3, 7, 6, 2, 1 ,14, 4, 11 ,15, 13, 9, 5 ],
[14 ,9 ,12 ,13 ,15 ,4 ,8 ,10 ,0, 2 ,1, 7 ,3, 11, 5, 6],
[12 ,11 ,0, 8 ,10 ,2 ,13 ,15, 5 ,4, 7 ,3, 6 ,9 ,14, 1],
[13, 8 ,14, 3, 9, 1, 0, 7, 15, 5, 4 ,10 ,12, 2, 6, 11],
[3 ,15, 2, 5 ,11, 6 ,4 ,7 ,12, 9, 1, 0, 13, 14, 10, 8],
[5 ,11 ,6 ,9 ,4 ,13 ,12, 0, 8, 2, 15, 10, 1, 7, 3, 14],
[5 ,0, 15, 8, 4, 6, 1 ,14, 10, 11 ,3, 9, 7, 12, 2, 13],
[15 ,14 ,6, 7 ,10, 1, 0, 11, 12, 8, 4, 9, 2, 5, 13, 3],
[11 ,14, 13, 1, 2, 3 ,12 ,4 ,15, 7, 9, 5 ,10, 6, 8, 0],
[6 ,13 ,3 ,2 ,11 ,9 ,5, 10, 1, 7, 12 ,14, 8, 4, 0, 15],
[4 ,6 ,12, 0 ,14 ,2 ,9, 13 ,11 ,8, 3, 15, 7, 10, 1, 5],
[8 ,10 ,9 ,11 ,14, 1, 7 ,15 ,13, 4, 0 ,12, 6 ,2 ,5 ,3],
[5 ,2 ,14 ,0 ,7 ,8, 6, 3, 11, 12, 13 ,15, 4 ,10, 9, 1],
[7 ,8 ,3 ,2 ,10 ,12, 4 ,6 ,11 ,13 ,5 ,15 ,0, 1 ,9 ,14],
[11 ,6, 14, 12 ,3, 5 ,1, 15, 8, 0, 10, 13, 9, 7, 4, 2],
[7 ,1, 2, 4 ,8 ,3 ,6 ,11 ,10 ,15 ,0 ,5 ,14 ,12 ,13 ,9],
[7 ,3, 1, 13, 12, 10, 5, 2, 8, 0, 6, 11, 14, 15, 4, 9],
[6 ,0 ,5 ,15 ,1 ,14 ,4 ,9 ,2 ,13 ,8 ,10, 11, 12, 7, 3],
[15 ,1, 3, 12, 4, 0, 6, 5, 2, 8, 14, 9, 13, 10, 7, 11],
[5, 7, 0 ,11 ,12, 1, 9, 10, 15, 6, 2, 3, 8, 4, 13, 14],
[12, 15, 11, 10 ,4, 5, 14, 0, 13, 7, 1, 2, 9, 8, 3, 6],
[6 ,14 ,10 ,5 ,15 ,8 ,7 ,1 ,3, 4 ,2 ,0 ,12, 9, 11, 13],
[14 ,13, 4 ,11, 15 ,8, 6, 9, 0, 7 ,3, 1, 2 ,10, 12, 5],
[14 ,4 ,0 ,10 ,6 ,5 ,1 ,3 ,9, 2 ,13 ,15 ,12 ,7, 8, 11],
[15 ,10, 8 ,3, 0 ,6, 9, 5, 1, 14 ,13, 11, 7 ,2 ,12, 4],
[0 ,13, 2, 4 ,12 ,14, 6 ,9 ,15, 1 ,10, 3 ,11 ,5 ,8 ,7],
[3 ,14, 13, 6, 4, 15, 8, 9 ,5 ,12, 10, 0, 2, 7 ,1, 11],
[0 ,1 ,9 ,7 ,11 ,13 ,5, 3, 14 ,12 ,4 ,2, 8 ,6 ,10, 15],
[11 ,0, 15, 8 ,13, 12, 3, 5, 10, 1, 4 ,6, 14, 9, 7, 2],
[13 ,0 ,9, 12, 11, 6, 3, 5 ,15, 8, 1 ,10, 4 ,14, 2, 7],
[14 ,10, 2 ,1 ,13, 9 ,8 ,11 ,7 ,3 ,6 ,12 ,15 ,5 ,4 ,0 ],
[12 ,3 ,9 ,1 ,4 ,5 ,10 ,2 ,6, 11 ,15, 0, 14, 7, 13 ,8],
[15 ,8 ,10, 7 ,0 ,12, 14 ,1, 5, 9, 6, 3, 13 ,11, 4, 2],
[4 ,7 ,13 ,10 ,1 ,2, 9, 6 ,12 ,8 ,14 ,5 ,3 ,0 ,11 ,15],
[6 ,0 ,5, 10, 11, 12, 9, 2, 1, 7, 4, 3, 14, 8, 13, 15],
[9 ,5 ,11, 10 ,13, 0, 2 ,1 ,8 ,6, 14 ,12 ,4 ,7 ,3 ,15],
[15 ,2, 12, 11 ,14, 13, 9, 5, 1, 3, 8, 7, 0, 10, 6, 4],
[11 ,1 ,7 ,4, 10 ,13 ,3, 8, 9 ,14 ,0, 15 ,6 ,5 ,2 ,12],
[5 ,4 ,7 ,1 ,11 ,12 ,14 ,15 ,10 ,13, 8 ,6 ,2 ,0 ,9 ,3],
[9 ,7 ,5 ,2, 14 ,15, 12 ,10 ,11 ,3 ,6 ,1, 8, 13, 0, 4],
[3 ,2, 7, 9, 0 ,15, 12 ,4, 6, 11, 5 ,14, 8 ,13, 10, 1],
[13 ,9 ,14, 6 ,12 ,8 ,1 ,2, 3 ,4 ,0, 7 ,5 ,10 ,11 ,15],
[5 ,7 ,11 ,8, 0, 14 ,9 ,13 ,10 ,12 ,3, 15, 6 ,1 ,4, 2],
[4 ,3 ,6 ,13 ,7 ,15 ,9 ,0 ,10 ,5 ,8 ,11 ,2 ,12 ,1, 14],
[1 ,7 ,15 ,14 ,2 ,6 ,4 ,9 ,12 ,11, 13, 3, 0, 8, 5, 10],
[9 ,14 ,5 ,7 ,8 ,15, 1 ,2 ,10, 4 ,13, 6 ,12 ,0 ,11, 3],
[0 ,11 ,3 ,12 ,5 ,2 ,1 ,9 ,8 ,10 ,14 ,15, 7 ,4 ,13, 6],
[7 ,15 ,4 ,0 ,10 ,9 ,2 ,5, 12 ,11 ,13 ,6 ,1 ,3 ,14 ,8],
[11 ,4, 0, 8 ,6 ,10 ,5 ,13 ,12 ,7 ,14, 3 ,1 ,2 ,9 ,15 ]]

In [16]:
import time
goal = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
h = []
i = 0
for start in starts:
  puz = EightPuzzle(start,goal,1.75,3)
  t1 = time.perf_counter()
  z = puz.solve()
  t2 = time.perf_counter()
  done = t2-t1
  print(i,z,done)
  h.append([z,done])
  i += 1

0 19858 52.2310155829764
1 29275 134.43247237498872
2 8083 6.748075791983865
3 54923 502.88626083300915
4 1525 0.37516354204853997
5 24403 89.91440504096681
6 14792 27.568633791990578
7 24038 87.86588950001169
8 17728 40.620576124987565
9 31893 172.0089954159921
10 4296 2.4159215000108816
11 3473 1.9608121669734828
12 20291 63.711621917027514
13 168857 7469.930666958971
14 4997 3.9098981249844655
15 4478 3.9446563749806955
16 62510 1026.6688927909709
17 3512 2.239650667004753
18 4726 3.9142947499640286
19 20185 102.21235708403401
20 8096 13.306384374969639
21 12412 34.147969375015236
22 8113 13.093279041000642
23 2738 1.367775124963373
24 8548 13.482217459008098
25 4619 3.932019791973289
26 64582 1162.6667636659695
27 3577 2.8346877500298433
28 156980 7038.297158540983
29 2714 1.440303582989145
30 1809 0.9073670829529874
31 12545 40.92493983299937
32 17053 78.0269237079774
33 19222 104.27085320901824
34 21524 118.6627599169733
35 10414 25.939072375011165
36 17461 78.6995824999758
37 12

In [17]:
import pandas as pd
df = pd.DataFrame(h)
df.to_csv("/Users/kirkduran/Documents/15_Puzzle_GWA_W1_75_3.csv", index = False)